# Introduction to Qoqo
Quantum Operation Quantum Operation  
Yes, we use [reduplication](https://en.wikipedia.org/wiki/Reduplication)

What Qoqo is
* A toolkit to represent quantum operations and circuits
* A thin runtime to run quantum measurements
* A way to serialize quantum circuits and measurement information
* A set of optional interfaces to devices, simulators and toolkits

What Qoqo is not
* A decomposer translating circuits to a specific set of gates
* A quantum circuit optimizer
* A collection of quantum algorithms


## A simple circuit and measurement

We show the construction of a simple entangling circuit and an observable measurement based on this circuit

### Entangling circuit snippet
Similar to many other toolkits the unitary entangling circuit can be constructed by adding operations to a circuit

In [10]:
from qoqo import Circuit
from qoqo import operations as ops

circuit_snippet = Circuit()
circuit_snippet += ops.Hadamard(qubit=0)
circuit_snippet += ops.CNOT(control=0, qubit=1)
print(circuit_snippet)
print(len(circuit_snippet))
print(circuit_snippet.get_operation_types())

Hadamard 0
CNOT 0 1

2
['Hadamard', 'CNOT']


### Measuring qubits
Qoqo uses classical registers for the readout. We need to add a classical register definition to the circuit and a measurement statement.
The number of projective measurements can be directly set in the circuit.  
The simulation and measurement of the circuit is handled by the qoqo_pyquest interface (in this example).

In [11]:
from qoqo_pyquest import PyQuestBackend
from qoqo import Circuit
from qoqo import operations as ops

circuit = Circuit()
circuit += ops.Definition(name='ro', vartype='bit', length=2, is_output=True)
circuit += ops.Hadamard(qubit=0)
circuit += ops.CNOT(control=0, qubit=1)
circuit += ops.PragmaRepeatedMeasurement(readout='ro', number_measurements=10)
print(circuit)
backend = PyQuestBackend(circuit=circuit, number_qubits=2)
result = backend.run()
for single_projective_measurement in result['ro'].register:
    print(single_projective_measurement)

Definition(False,True) ro BIT[2]
Hadamard 0
CNOT 0 1
PragmaRepeatedMeasurement(10) ALL ro

[1. 1.]
[0. 0.]
[1. 1.]
[1. 1.]
[0. 0.]
[1. 1.]
[1. 1.]
[0. 0.]
[0. 0.]
[0. 0.]


### Measuring Observables
Qoqo includes the direct evaluation of projective measurements to an observable measurement e.g. 3 * < Z0 > + < Z0 Z1 >  
The measurement is defined by a set of expectation values of a product of pauli operators and a matrix that combines the expectation values  

In [12]:
from qoqo.measurements import BRMeasurementInput, BasisRotationMeasurement
from qoqo import DoUnitary
from qoqo_pyquest import PyQuestBackend
from qoqo import Circuit
from qoqo import operations as ops
import numpy as np
import scipy.sparse as sp

circuit = Circuit()
circuit += ops.Definition(name='ro', vartype='bit', length=2, is_output=True)
circuit += ops.Hadamard(qubit=0)
circuit += ops.CNOT(control=0, qubit=1)
circuit += ops.PragmaRepeatedMeasurement(readout='ro', number_measurements=10)

measurement_input = BRMeasurementInput(
    pauli_product_qubit_masks={'ro': {0: [0], 1: [0, 1]}}, # From readout 'ro' measure two pauli products 0: < Z0 > and 1: < Z0 Z1 >
    pp_to_exp_val_matrix= np.array([[3, 1]]), # One expectation value: 3 * pauli_product0 + 1 * pauli_product1
    number_qubits=2,
    number_pauli_products=2,
    measured_exp_vals=['example'])

measurement = BasisRotationMeasurement(measurement_input=measurement_input, circuit_list=[circuit], verbose=False)

backend = PyQuestBackend(circuit=circuit, number_qubits=2)

do_unitary = DoUnitary(measurement=measurement, backend=backend, free_parameters=[]) # This quantum program does not have any free parameters
do_unitary()[0]

2.2

### De/Serializing the quantum program

In [14]:
from qoqo.measurements import BRMeasurementInput, BasisRotationMeasurement
from qoqo import DoUnitary
from qoqo_pyquest import PyQuestBackend
from qoqo import Circuit
from qoqo import operations as ops
import numpy as np
import scipy.sparse as sp
from hqsbase.qonfig import Qonfig

circuit = Circuit()
circuit += ops.Definition(name='ro', vartype='bit', length=2, is_output=True)
circuit += ops.Hadamard(qubit=0)
circuit += ops.CNOT(control=0, qubit=1)
circuit += ops.PragmaRepeatedMeasurement(readout='ro', number_measurements=10)

measurement_input = BRMeasurementInput(
    pauli_product_qubit_masks={'ro': {0: [0], 1: [0, 1]}}, # From readout 'ro' measure two pauli products 0: < Z0 > and 1: < Z0 Z1 >
    pp_to_exp_val_matrix= np.array([[3,1]]), # One expectation value: 3 * pauli_product0 + 1 * pauli_product1
    number_qubits=2,
    number_pauli_products=2,
    measured_exp_vals=['example'])

measurement = BasisRotationMeasurement(measurement_input=measurement_input, circuit_list=[circuit], verbose=False)
measurement_json = measurement.to_qonfig().to_json()

measurement = Qonfig.from_json(measurement_json).to_instance()

#print(measurement_json)

## Fine control over decoherence
Qoqo allows full control over decoherence by placing decoherence operations in the circuit on the same level as gates.  
Example: Letting only one qubit decay.  
The backend automatically switches from statevector simulation to density matrix simulation in the presence of noise.

In [18]:
from qoqo_pyquest import PyQuestBackend
from qoqo import Circuit
from qoqo import operations as ops

damping = 0.1
number_measurements = 100000
circuit = Circuit()
circuit += ops.Definition(name='ro', vartype='bit', length=2, is_output=True)
circuit += ops.PauliX(qubit=0)
circuit += ops.PauliX(qubit=1)
circuit += ops.PragmaDamping(qubit=0, gate_time=1, rate=damping)
circuit += ops.PragmaRepeatedMeasurement(readout='ro', number_measurements=number_measurements)
print(circuit)
backend = PyQuestBackend(circuit=circuit, number_qubits=2)
result = backend.run()
sum_test = np.array([0.0, 0.0])
for single_projective_measurement in result['ro'].register:
    #print(single_projective_measurement)
    sum_test += single_projective_measurement
sum_test/number_measurements


Definition(False,True) ro BIT[2]
PauliX 0
PauliX 1
PragmaDamping(1e0, 1e-1) 0
PragmaRepeatedMeasurement(100000) ALL ro



array([0.90556, 1.     ])

## Symbolic parameters
In many cases, operation parameters depend on a symbolic parameter of the whole quantum program (time in time-evolution, overrotation, variational parameters...)  
Qoqo allows the fast calculation of symbolic parameter expressions.  
Expressions are provided in string form.  
DoUnitary can automatically replace symbolic parameters using call parameters.

### Writing the symbolic circuit and replacing symbolic parameters

In [20]:
from qoqo import Circuit
from qoqo import operations as ops
circuit = Circuit()
print('Symbolic circuit')
circuit += ops.RotateX(qubit=0, theta='3*time+offset')

print(circuit)

circuit.substitute_parameters({'time': 1/3, 'offset':1})
print('After substitution')
print(circuit)


Symbolic circuit
RotateX(3*time+offset) 0

After substitution
RotateX(2e0) 0



### Symbolic parameters in a full quantum program

In [22]:
from qoqo.measurements import BRMeasurementInput, BasisRotationMeasurement
from qoqo import DoUnitary
from qoqo_pyquest import PyQuestBackend
from qoqo import Circuit
from qoqo import operations as ops
import numpy as np
import scipy.sparse as sp

number_measurements = 100000

circuit = Circuit()
circuit += ops.Definition(name='ro', vartype='bit', length=2, is_output=True)
circuit += ops.RotateX(qubit=0, theta='3*time+offset')
circuit += ops.PragmaRepeatedMeasurement(readout='ro', number_measurements=number_measurements)

measurement_input = BRMeasurementInput(
    pauli_product_qubit_masks={'ro': {0: [0]}}, # From readout 'ro' measure one pauli product 0: < Z0 > 
    pp_to_exp_val_matrix= np.array([[1]]), # One expectation value: 1* pauli_product0
    number_qubits=1,
    number_pauli_products=1,
    measured_exp_vals=['example'])

measurement = BasisRotationMeasurement(measurement_input=measurement_input, circuit_list=[circuit], verbose=False)

backend = PyQuestBackend(circuit=circuit, number_qubits=2)

do_unitary = DoUnitary(measurement=measurement, backend=backend, free_parameters=['time', 'offset']) # The symbolic parameter is the free parameter
do_unitary([0.5, 0])

exp_val_example             0.0706
unitary_parameter_time      0.5000
unitary_parameter_offset    0.0000
dtype: float64

## Testing scaling performance with qoqo_mock
Quantum simulators cannot simulate systems with a significant number of qubits fast enough to benchmark qoqo with a large number of qubits and operations.
The qoqo_mock interface can be used to benchmark qoqo without simulating a quantum computer.

In [23]:
from qoqo.measurements import BRMeasurementInput, BasisRotationMeasurement
from qoqo import DoUnitary
from qoqo_mock import MockedBackend
from qoqo import Circuit
from qoqo import operations as ops
import numpy as np
import timeit

number_measurements = 10000
number_operations = 1000000
number_qubits = 500

circuit = Circuit()

circuit += ops.Definition(name='ro', vartype='bit', length=number_qubits, is_output=True)

for i, q in zip(np.random.randint(0,4,number_operations), np.random.randint(0,500,number_operations)):
    if i == 0:
        circuit += ops.RotateX(qubit=q, theta="4*theta_x")
    if i == 1:
        circuit += ops.RotateY(qubit=q, theta="2*theta_y")
    if i == 2:
        circuit += ops.RotateZ(qubit=q, theta="3*theta_z")
    if i == 4:
        circuit += ops.ControlledPauliZ(qubit=q, control=0)
circuit += ops.PragmaRepeatedMeasurement(readout='ro', number_measurements=number_measurements)

pp_dict = dict()
pp_dict[number_qubits] = list()
for i in range(number_qubits):
    pp_dict[i] = [i]

measurement_input = BRMeasurementInput(
    pauli_product_qubit_masks={'ro': pp_dict},
    pp_to_exp_val_matrix= np.ones((1,number_qubits+1)),
    number_qubits=number_qubits,
    number_pauli_products=number_qubits+1,
    measured_exp_vals=['example'])

measurement = BasisRotationMeasurement(measurement_input=measurement_input, circuit_list=[circuit], verbose=False)
backend= MockedBackend(number_qubits=number_qubits, mocked_qubits=number_qubits)
do_unitary = DoUnitary(measurement=measurement, backend=backend, free_parameters=['theta_x', 'theta_y', 'theta_z'])
print(do_unitary([0,1,2]))
timeit.timeit('do_unitary([0,1,2])', globals=globals(),number=1)

exp_val_example              1.0434
unitary_parameter_theta_x    0.0000
unitary_parameter_theta_y    1.0000
unitary_parameter_theta_z    2.0000
dtype: float64


4.043968541009235